# Escaping the maze
In this notebook we will cover the basics of a reinforcement learning (RL) environment.

Specifically, we will cover the observation, action, and state space following the example of a maze.

In [ ]:
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

from qgym.environment import Environment
from qgym.rewarder import Rewarder

## Map of the maze

Our maze will have 4 different field types.

- `S`: start position
- `F`: a free field
- `W`: a wall
- `G`: the goal

In [ ]:
maze_map_4x4 = ["FSFF", "SWFW", "FFFW", "WFFG"]

- `0`: UP
- `1`: RIGHT
- `2`: DOWN
- `3`: LEFT

In [ ]:
class MazeRewarder(Rewarder):
    def compute_reward(self, old_state, action, new_state):
        row, col = new_state["position"]
        if new_state["maze_map"][row][col] == b"G":
            return 1
        else:
            return 0

In [ ]:
class Maze(Environment):
    def __init__(self, maze_map):
        maze_map = np.asarray(maze_map, dtype="c")

        self.nrows = maze_map.shape[0]
        self.ncols = maze_map.shape[1]

        self.start_position_distribution = (maze_map == b"S").ravel().astype("float64")
        self.start_position_distribution /= self.start_position_distribution.sum()

        self.action_space = gym.spaces.Discrete(4)  # {0,1,2,3}
        self.observation_space = gym.spaces.Discrete(self.nrows * self.ncols)
        self._state = {"position": None, "maze_map": maze_map}
        self._rewarder = MazeRewarder()

    def rowcol_to_pos(self, row, col):
        return row * self.nrows + col

    def pos_to_rowcol(self, pos):
        return int(pos / self.nrows), pos % self.nrows

    def reset(self, *, seed=None, return_info=False):
        start_position = self.rng.choice(
            self.nrows * self.ncols, p=self.start_position_distribution
        )
        self._state["position"] = self.pos_to_rowcol(start_position)

        return super().reset(seed=seed, return_info=return_info)

    def _update_state(self, action):
        row, col = self._state["position"]

        # compute new position
        if action == 0:  # up
            row = max(row - 1, 0)
        elif action == 1:  # right
            col = min(col + 1, self.ncols - 1)
        elif action == 2:  # down
            row = min(row + 1, self.nrows - 1)
        elif action == 3:  # left
            col = max(col - 1, 0)
        else:
            raise ValueError("Invalid action supplied.")

        # go to new position if it is not a wall
        if self._state["maze_map"][row][col] != b"W":
            self._state["position"] = (row, col)
        # else we stay where we are

    def _obtain_observation(self):
        return self.rowcol_to_pos(*self._state["position"])

    def _is_done(self):
        row, col = self._state["position"]
        return self._state["maze_map"][row][col] == b"G"

    def _obtain_info(self):
        return {}

    def _compute_reward(self, old_state, action):
        return super()._compute_reward(
            old_state=old_state, action=action, new_state=self._state
        )

In [ ]:
# todo: run script
env = Maze(maze_map_4x4)
check_env(env)

model = PPO("MlpPolicy", env, verbose=1)

model.learn(int(1e5))